<a href="https://colab.research.google.com/github/parththegoat1234/MoneyBall_Project/blob/main/Moneyball_Project_Section_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="#de3023"><h1><b>REMINDER: MAKE A COPY OF THIS NOTEBOOK, DO NOT EDIT</b></h1></font>

# Notebook 3: Expanding on our budget optimization through cluster analysis

In this notebook we will:
* Learn about cluster analysis and how it can be used in sports analytics
* Learn about pitching and fielding data
* Use machine learning to cluster players
* Along with NB2, identify undervalued players who we could replace our current players with (because they're aging out, are too expensive, are leaving, etc.)


In [ ]:
#@title Run this to download data and prepare our environment! { display-mode: "form" }
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import IntSlider, Output

# Rename columns to be more descriptive
abbrev_map = {
    # General
    "W": "Wins",
    "L": "Losses",
    "G": "Games",
    "GS": "GamesStarted",

    # Batting
    "R": "Runs",

    "AB": "AtBats",

    "H": "Hits",
    "2B": "Doubles",
    "3B": "Triples",
    "HR": "HomeRuns",

    "BB": "Walks",
    "HBP": "HitsByPitch",
    "SF": "SacrificeFlies",

    # Fielding
    "PO": "Putouts",
    "A": "Assists",
    "E": "Errors",
    "DP": "DoublePlays",
    "POS": "Position",

    "PB": "PassedBalls",
    "WP": "WildPitch",
    "SB": "StolenBasesAllowed",
    "CS": "CaughtStealing",
    "ZR": "ZoneRating",

    # Additional
    "CG": "CompleteGames",
    "SHO": "Shutouts",
    "SV": "Saves",
    "IPouts": "OutsPitched",
    "ER": "EarnedRuns",
    "SO": "Strikeouts",
    "BAOpp": "OpponentsBattingAverage",
    "ERA": "EarnedRunAverage",
    "IBB": "IntentionalWalks",
    "BK": "Balks",
    "BFP": "BattersFacedByPitcher",
    "GF": "GamesFinished",
    "SH": "SacrificeHits",
    "SF": "SacrificeFlies",
    "GIDP": "GroundedIntoDoublePlay"
}

def process(df):
  df = df.loc[df["yearID"] > 1972]
  df = df.rename(columns=abbrev_map)

  return df

# BATTING
batting = pd.read_csv("https://drive.google.com/uc?id=17IM9kxyscGEpu-4Ov5p7MpCKz9UVHLla")
batting = process(batting)
batting = batting.loc[batting["AtBats"] > 130] # remove rookies' rows
batting = batting.drop(columns=['teamID', 'lgID'])

# FIELDING
fielding = pd.read_csv("https://drive.google.com/uc?id=1Vhh5hMTWmgHTQ7lLUL1ugWNIeHIt4Mev")
fielding = process(fielding)
fielding = fielding.drop(columns=['WildPitch', 'GamesStarted', 'CaughtStealing', 'InnOuts', 'teamID', 'lgID']) # WildPitch has only NaNs for data since 1972, CaughtStealing only relevant for catchers

# PITCHING
pitching = pd.read_csv("https://drive.google.com/uc?id=11QmRiKddAVeSt6gCtW-5r0Qp_bFnxOVb")
pitching = process(pitching)
pitching = pitching.drop(columns=['WildPitch', 'GamesStarted', 'Wins', 'Losses', 'BattersFacedByPitcher', 'teamID', 'lgID']) # WildPitch has only NaNs for data since 1972

# Milestone 1: Importance of Cluster Analysis in Data Analytics

Cluster analysis is a fundamental technique in data science used to uncover natural groupings or clusters within a dataset. It plays a crucial role in various aspects of data analytics, including the following:

1. **Data Exploration**: Clustering facilitates the exploration of large and complex datasets by summarizing the data into meaningful groups.

2. **Feature Engineering**: Clustering can be used for creating new features based on cluster assignments. These engineered features can improve models by capturing structures and relationships in the data.

3. **Anomaly Detection**: Clustering can also be utilized for anomaly detection by identifying data points that do not conform to the typical patterns observed in the data. Anomalies may represent unusual events, outliers, or errors that warrant further investigation.



### 💡 Discussion Question

What are some examples of clustering analysis you may have seen, in this class or otherwise?

### How Cluster Analysis Can Apply to Sports

In sports analytics, clustering analysis holds particular significance due to its ability to uncover insights and patterns within player performance, team dynamics, and game strategies:

1. **Player/Team Comparison**: Clustering enables comparisons between players or teams based on their characteristics, performance metrics, playing styles, etc. By clustering players or teams into meaningful groups, analysts can identify similarities, differences, strengths, and weaknesses, aiding coaches, managers, and analysts in strategic decision-making. This is what you'll be doing in this notebook!

2. **Player Development**: Clustering can also aid in player development efforts by identifying players with similar skill sets or playing styles. Coaches and talent scouts can use clustering insights to personalize training programs, match players with compatible teammates, or identify potential recruits based on desired characteristics.

3. **Fan Engagement**: Clustering analysis can be applied to fan segmentation, allowing sports organizations to understand fan preferences, behaviors, and engagement levels.



### 💡 Discussion Question

Can you think of any other uses for clustering analysis in sports?


# Milestone 2: Introducing the `fielding` and `pitching` datasets

## Fielding Statistics

On our journey through baseball statistics, we focus on a select group of fielding metrics to make our analysis approachable, especially for those new to the game's data. This narrowed focus allows us to explore the defensive side of baseball effectively. Remember, you're encouraged to dive deeper into the dataset on your own!

We'll start with the following columns: `playerID`, `yearID`, `Positions`, and key defensive metrics such as `Putouts`, `Assists`, `Errors`, `DoublePlays`, `StolenBasesAllowed`, and `ZoneRating`. These statistics offer a well-rounded picture of how players contribute defensively.


<!--

#### Putouts
The **Putouts** statistic refers to the number of times a fielder catches a batted or thrown ball and records an out. It's a fundamental measure of defensive performance, indicating how effectively a fielder contributes to retiring opposing batters.

#### Assists
**Assists** represent the number of times a fielder throws the ball to another player to record an out. This statistic highlights a fielder's ability to contribute to outs by executing accurate throws to teammates covering bases or other fielding positions.

#### Errors
**Errors** occur when a fielder fails to make a play that should have been made with ordinary effort, resulting in the advancement of a baserunner or the continuation of an at-bat. It's an important metric for evaluating defensive proficiency, as it reflects lapses in fielding execution.

#### Double Plays
**Double Plays** occur when the defense records two outs during a single play. This statistic is crucial for evaluating a team's ability to efficiently transition from defense to offense and halt opposing scoring opportunities.

#### Stolen Bases Allowed
**Stolen Bases Allowed** represent instances when a baserunner successfully advances to the next base while the pitcher is throwing the ball to home plate. This metric reflects a pitcher's and catcher's ability to deter baserunners from attempting to steal bases effectively.

#### Caught Stealing
**Caught Stealing** refers to instances when a baserunner attempts to steal a base but is tagged out before reaching the base safely. This statistic demonstrates the defensive team's capability to thwart baserunning attempts and prevent opposing runners from advancing.

#### Zone Rating
**Zone Rating** is a sabermetric statistic that measures a fielder's defensive ability within their assigned fielding zone. It evaluates the number of plays made by a fielder compared to the number of opportunities they have in their defensive area, providing insight into their range and reliability in fielding balls hit in their vicinity.

These fielding statistics play a significant role in assessing a team's defensive performance and individual players' contributions on the field. They provide valuable insights into defensive strengths and areas for improvement, helping teams optimize their defensive strategies and player deployment.

-->

In [ ]:
fielding.head()

,playerID,yearID,stint,Position,Games,Putouts,Assists,Errors,DoublePlays,PassedBalls,StolenBasesAllowed,ZoneRating
69476,aaronha01,1973,1,OF,105,206,5,5.0,0,NaN,NaN,NaN
69477,abbotgl01,1973,1,P,5,1,2,0.0,0,NaN,NaN,NaN
69478,acostcy01,1973,1,P,48,2,10,2.0,2,NaN,NaN,NaN
69479,adamsmi02,1973,1,OF,24,45,0,1.0,0,NaN,NaN,NaN
69480,ageeto01,1973,1,OF,67,114,2,2.0,2,NaN,NaN,NaN


### ✍ Exercise: One-Hot Encoding Player Positions

In machine learning, algorithms usually require numerical input. However, player positions are categorical (like "SS" for shortstop or "CF" for center fielder), which poses a challenge. **One-hot encoding** is a technique that converts these categories into a numerical format that algorithms can work with. This transformation is crucial for performing more advanced analyses, such as nearest neighbors, where we compare players based on their stats and positions.

**Your Task:** Convert the `Position` column in `fielding` to a numerical format using one-hot encoding. This will allow us to include player positions in our machine learning models, enabling more robust analysis.

- **Hint:** Use `pd.get_dummies()` to perform one-hot encoding on the `Position` column.


In [ ]:
fielding = pd.get_dummies(fielding, columns=['Position'])
fielding.head()

,playerID,yearID,stint,Games,Putouts,Assists,Errors,DoublePlays,PassedBalls,StolenBasesAllowed,ZoneRating,Position_1B,Position_2B,Position_3B,Position_C,Position_OF,Position_P,Position_SS
69476,aaronha01,1973,1,105,206,5,5.0,0,NaN,NaN,NaN,False,False,False,False,True,False,False
69477,abbotgl01,1973,1,5,1,2,0.0,0,NaN,NaN,NaN,False,False,False,False,False,True,False
69478,acostcy01,1973,1,48,2,10,2.0,2,NaN,NaN,NaN,False,False,False,False,False,True,False
69479,adamsmi02,1973,1,24,45,0,1.0,0,NaN,NaN,NaN,False,False,False,False,True,False,False
69480,ageeto01,1973,1,67,114,2,2.0,2,NaN,NaN,NaN,False,False,False,False,True,False,False


### Discussion Question

Why couldn't we just assign a number for each position (like 0 for pitchers, 1 for first baseman, etc.)?

*Hint*: how might this affect our calculations when finding clusters and nearest neighbors?

## Pitching Statistics

Our exploration of baseball analytics brings us to pitching statistics, essential for understanding a pitcher's contribution to their team. This section highlights a selection of fundamental metrics, providing a basis for assessing pitcher performance.

### Featured Metrics

- **Earned Run Average (ERA):** The average number of earned runs a pitcher allows per nine innings, a primary indicator of pitching effectiveness.
- **Strikeouts (SO):** Reflects the number of batters a pitcher retires by strikeout, showcasing their ability to overpower hitters.
- **Saves (SV):** Important for relief pitchers, this metric indicates how often a pitcher successfully preserves a lead, securing a win.
- **Complete Games (CG):** Shows the number of games a pitcher pitches from start to finish, highlighting durability and stamina.
- **Opponents Batting Average (BAOpp):** The batting average of opposing hitters, providing insights into a pitcher's ability to limit hits.

For a comprehensive understanding of all metrics used to evaluate pitchers, feel free to take a look at https://www.mlb.com/glossary/!. These additional metrics, including Shutouts (SHO), Outs Pitched (IPouts), and Runs Allowed (R), among others, offer deeper insights into various aspects of pitching performance.

By focusing on these key statistics, we can begin to unravel the complexities of pitching in baseball, assessing how individual pitchers contribute to their teams' defensive efforts.

<!--
#### Complete Games (CG)
**Complete Games** represent the number of games a pitcher completes by pitching the entire game. It showcases a pitcher's durability and ability to pitch effectively throughout an entire game without being replaced by a relief pitcher.

#### Shutouts (SHO)
**Shutouts** indicate the number of games in which a pitcher pitches a complete game without allowing the opposing team to score. It demonstrates a pitcher's dominance on the mound and their ability to stifle opposing offenses.

#### Saves (SV)
**Saves** refer to the number of times a relief pitcher successfully preserves a lead for their team. It's a crucial statistic for evaluating a relief pitcher's effectiveness in closing out games and securing victories.

#### Outs Pitched (IPouts)
**Outs Pitched** represent the total number of outs recorded by a pitcher. It's an essential metric for assessing a pitcher's workload and effectiveness in getting batters out.

#### Earned Runs (ER)
**Earned Runs** denote the number of runs that were scored without the aid of errors or passed balls by the defense. It reflects a pitcher's performance in preventing opposing teams from scoring runs.

#### Strikeouts (SO)
**Strikeouts** indicate the number of times a pitcher strikes out a batter. It showcases a pitcher's ability to overpower hitters and record outs without allowing the ball to be put in play.

#### Opponents Batting Average (BAOpp)
**Opponents Batting Average** is the batting average of opposing batters against the pitcher. It provides insights into how effectively a pitcher limits opposing hitters' ability to get hits.

#### Earned Run Average (ERA)
**Earned Run Average** is the average number of earned runs a pitcher allows per nine innings pitched. It's a key metric for evaluating a pitcher's effectiveness in preventing runs from scoring.

#### Intentional Walks (IBB)
**Intentional Walks** are walks issued to a batter intentionally, typically to set up a force play or to face a weaker hitter. It's a strategic move by the pitcher and team to control the situation on the field.

#### Balks (BK)
**Balks** occur when a pitcher makes an illegal movement or action while on the pitching rubber, resulting in a penalty. It's an infraction that can result in baserunners advancing.

#### Batters Faced by Pitcher (BFP)
**Batters Faced by Pitcher** represent the total number of batters faced by a pitcher. It's a comprehensive measure of a pitcher's workload and their impact on the game.

#### Games Finished (GF)
**Games Finished** denote the number of games in which a pitcher enters the game after the starting pitcher and completes the game. It reflects a relief pitcher's ability to effectively close out games.

#### Runs Allowed (R)
**Runs Allowed** indicate the total number of runs allowed by a pitcher. It's a critical statistic for assessing a pitcher's performance in preventing opposing teams from scoring.

#### Sacrifice Hits (SH)
**Sacrifice Hits** occur when a batter purposely bunts the ball to advance a baserunner. It's a strategic offensive move employed by hitters to advance runners into scoring position.

#### Grounded Into Double Play (GIDP)
**Grounded Into Double Play** refers to when a batter hits the ball in a way that allows the defense to record two outs. It reflects both the batter's and pitcher's ability to induce ground balls and turn them into double plays.

These pitching statistics provide valuable insights into a pitcher's performance and contribution to their team's success on the field.
-->


In [ ]:
pitching.head()

,playerID,yearID,stint,Games,CompleteGames,Shutouts,Saves,OutsPitched,Hits,EarnedRuns,...,OpponentsBattingAverage,EarnedRunAverage,IntentionalWalks,HitsByPitch,Balks,GamesFinished,Runs,SacrificeHits,SacrificeFlies,GroundedIntoDoublePlay
20318,abbotgl01,1973,1,5,1,0,0,56,16,8,...,0.225,3.86,0.0,0.0,1,1,8,0.0,1.0,0.0
20319,acostcy01,1973,1,48,0,0,18,291,66,24,...,0.194,2.23,3.0,7.0,1,42,30,6.0,2.0,4.0
20320,akerja01,1973,1,47,0,0,12,191,76,29,...,0.308,4.10,6.0,2.0,0,32,33,8.0,1.0,10.0
20321,alburvi01,1973,1,14,0,0,0,70,13,7,...,0.169,2.70,2.0,0.0,1,5,7,0.0,0.0,5.0
20322,alexado01,1973,1,29,10,0,0,524,169,75,...,0.258,3.86,5.0,7.0,0,2,85,7.0,5.0,23.0


# Milestone 3: More Processing!

### Handling Multiple Stints

Sometimes players play on different teams in a given year! In order to summarize a given player's performance per year, we will alter our dataframe to have just one number per metric per `playerID`, `yearID` combination. Run the cell below to combine the data in this way! Take a look at the methods used and try to understand how this accomplishes our task!


In [ ]:
batting = batting.groupby(['playerID', 'yearID']).sum(numeric_only=True)
batting = batting.drop(columns=['stint'])
batting = batting.reset_index()

fielding = fielding.groupby(['playerID', 'yearID']).sum(numeric_only=True)
fielding = fielding.drop(columns=['stint'])
fielding = fielding.reset_index()

pitching = pitching.groupby(['playerID', 'yearID']).sum(numeric_only=True)
pitching = pitching.drop(columns=['stint'])
pitching = pitching.reset_index()

### ✍ Exercise: Normalizing by Number of Games

Since we're comparing players no matter the situation in a given season, we need to make sure we're comparing them fairly! Just as we did with the `WinningPercentage` in the first notebook, here we need to normalize all of the columns by the number of games each player has played in a given season.

Because some statistics correspond to rates, though, it won't make sense to normalize them by the number of games. Why might that be? What's happening in the below code?



In [ ]:
battingRateStatistics = pd.DataFrame({"playerID": batting["playerID"],
                                      "yearID": batting["yearID"],
                                      "BattingAverage": batting.eval("Hits / AtBats"),
                                      "OnBasePercentage": batting.eval("(Hits + Walks + HitsByPitch) / (AtBats + Walks + HitsByPitch + SacrificeFlies)"),
                                      "SluggingPercentage": batting.eval("(Hits + Doubles + 2*Triples + 3*HomeRuns) / AtBats")})

pitchingRateStatistics = pitching[["playerID", "yearID", "OpponentsBattingAverage"]]
pitching = pitching.drop(columns=["OpponentsBattingAverage"])

fieldingRateStatistics = fielding[["playerID", "yearID", "ZoneRating", "Position_1B", "Position_2B", "Position_3B", "Position_C", "Position_OF", "Position_P", "Position_SS"]]
fielding = fielding.drop(columns=["ZoneRating", "Position_1B", "Position_2B", "Position_3B", "Position_C", "Position_OF", "Position_P", "Position_SS"])

Now that we've saved those rate statistics to the side, we can use the `.div()` method to divide all of our columns by another. Here's the template:

```python
NORMALIZED_DATA = DATA.div(COLUMN, axis=0)
```

Note that this divides ALL of our columns though! You should save the normalized data to a new dataframe (as we've done above), and, for whichever columns that don't make sense to normalize by `Games`, be sure to replace them using the original data!

Lastly, remove the `Games` column using the `.drop()` method, since we will no longer need this column in our analysis!


In [ ]:
batting_normalized = batting.drop(columns=['playerID']).div(batting['Games'], axis=0)
batting_normalized = batting_normalized.drop(columns=['Games'])
batting_normalized[['playerID', 'yearID']] = batting[['playerID', 'yearID']]

fielding_normalized = fielding.drop(columns=['playerID']).div(fielding['Games'], axis=0)
fielding_normalized = fielding_normalized.drop(columns=['Games'])
fielding_normalized[['playerID', 'yearID']] = fielding[['playerID', 'yearID']]

pitching_normalized = pitching.drop(columns=['playerID']).div(pitching['Games'], axis=0)
pitching_normalized = pitching_normalized.drop(columns=['Games'])
pitching_normalized[['playerID', 'yearID']] = pitching[['playerID', 'yearID']]

batting_normalized

,yearID,AtBats,Runs,Hits,Doubles,Triples,HomeRuns,RBI,StolenBasesAllowed,CaughtStealing,Walks,Strikeouts,IntentionalWalks,HitsByPitch,SacrificeHits,SacrificeFlies,GroundedIntoDoublePlay,playerID
0,1973,3.266667,0.700000,0.983333,0.100000,0.008333,0.333333,0.800000,0.008333,0.008333,0.566667,0.425000,0.108333,0.008333,0.000000,0.033333,0.058333,aaronha01
1,1974,3.035714,0.419643,0.812500,0.142857,0.000000,0.178571,0.616071,0.008929,0.000000,0.348214,0.258929,0.053571,0.000000,0.008929,0.017857,0.053571,aaronha01
2,1975,3.394161,0.328467,0.795620,0.116788,0.014599,0.087591,0.437956,0.000000,0.007299,0.510949,0.372263,0.021898,0.007299,0.007299,0.043796,0.109489,aaronha01
3,1976,3.188235,0.258824,0.729412,0.094118,0.000000,0.117647,0.411765,0.000000,0.011765,0.411765,0.447059,0.011765,0.000000,0.000000,0.023529,0.094118,aaronha01
4,1998,2.741573,0.370787,0.764045,0.157303,0.011236,0.134831,0.460674,0.033708,0.033708,0.101124,0.314607,0.011236,0.000000,0.022472,0.056180,0.022472,abbotje01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17812,2019,2.955556,0.333333,0.488889,0.111111,0.011111,0.100000,0.355556,0.000000,0.000000,0.222222,1.088889,0.000000,0.033333,0.000000,0.000000,0.044444,zuninmi01
17813,2021,3.055046,0.587156,0.660550,0.100917,0.018349,0.302752,0.568807,0.000000,0.000000,0.311927,1.211009,0.000000,0.064220,0.000000,0.009174,0.064220,zuninmi01
17814,1992,3.161290,0.370968,0.870968,0.153226,0.008065,0.024194,0.346774,0.016129,0.016129,0.201613,0.483871,0.008065,0.032258,0.056452,0.032258,0.048387,zupcibo01
17815,1993,2.028369,0.283688,0.489362,0.170213,0.014184,0.014184,0.184397,0.035461,0.014184,0.191489,0.382979,0.014184,0.014184,0.056738,0.021277,0.049645,zupcibo01


## Merging

Since we eventually want to have a complete view of each player, including their batting, fielding, and pitching statistics, we need to make a new table that includes all this info!

This means we'll be merging more datasets like we did for the previous notebook. For your reference, this was the code we used last time:

```python
merged = pd.merge(batting, salaries, on=["playerID", "yearID"])
```

Note that this code matched the rows in the two datasets that had the exact same `playerID` and `yearID` and made a new row containing all of the information from both rows in the new dataframe. When you merge two tables this way, it's called an inner join.





### 💡 Discussion Question

With our batting, pitching and fielding datasets, though, an inner join won't make as much sense! Why might that be?

In our case, we want to use an outer join instead, which will include every `'playerID', 'yearID'` combination, and pull data that matches these combinations from those dataframes. If the combination isn't found in a given dataframe, then the merging will place in `NaN` values for the corresponding columns. This stands for "Not a Number" and is a value typically used for missing data or any undefined numbers (like $\frac{0}{0}$).

We can use the example code we have above and add in the parameter `how='outer'` to specify we want an outer join, and the `suffixes` parameter as a list of strings we'd like to add to the end of column names in the event we have columns in common between the merged datasets (like how `Walks` appears in both the batting and pitching datasets but have very different meanings). What could you make these suffixes to describe the data best?

In the code cell below try to figure out how you could merge all three datasets!



In [ ]:
merged = pd.merge(pitching_normalized, batting_normalized, on=['playerID', 'yearID'], how='outer', suffixes=['AsPitcher', 'AsBatter'])
merged = pd.merge(merged, fielding_normalized, on=['playerID', 'yearID'], how='outer', suffixes=['AsBatter', 'AsFielder'])
merged.head()

,yearID,CompleteGames,Shutouts,Saves,OutsPitched,HitsAsPitcher,EarnedRuns,HomeRunsAsPitcher,WalksAsPitcher,StrikeoutsAsPitcher,...,HitsByPitchAsBatter,SacrificeHitsAsBatter,SacrificeFliesAsBatter,GroundedIntoDoublePlayAsBatter,Putouts,Assists,Errors,DoublePlays,PassedBalls,StolenBasesAllowedAsFielder
0,2004,0.0,0.0,0.000000,2.909091,1.818182,0.727273,0.090909,0.909091,0.454545,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.00,0.000000,0.0,0.0
1,2006,0.0,0.0,0.000000,3.533333,0.911111,0.533333,0.200000,0.622222,1.088889,...,NaN,NaN,NaN,NaN,0.022222,0.111111,0.00,0.022222,0.0,0.0
2,2007,0.0,0.0,0.000000,3.880000,1.560000,0.920000,0.160000,0.680000,1.440000,...,NaN,NaN,NaN,NaN,0.080000,0.160000,0.04,0.000000,0.0,0.0
3,2008,0.0,0.0,0.000000,3.106383,1.042553,0.638298,0.085106,0.744681,1.042553,...,NaN,NaN,NaN,NaN,0.063830,0.127660,0.00,0.000000,0.0,0.0
4,2009,0.0,0.0,0.520548,2.931507,0.671233,0.273973,0.054795,0.465753,1.095890,...,NaN,NaN,NaN,NaN,0.027397,0.068493,0.00,0.013699,0.0,0.0


Lastly, let's add those rate statistics back in with one last merge in the code cell below

In [ ]:
merged = pd.merge(merged, battingRateStatistics, on=['playerID', 'yearID'], how='outer')
merged = pd.merge(merged, pitchingRateStatistics, on=['playerID', 'yearID'], how='outer')
merged = pd.merge(merged, fieldingRateStatistics, on=['playerID', 'yearID'], how='outer')

## Getting one set of values per player



One last bit of processing! Since players potentially play for multiple years, we want to have some way of having one set of values to describe a particular player.


### Option 1: Focus on players' average performance

Take a look at the previous exercises to see how you can use `.groupby()` to get players' average performance! If you'd rather focus on players' most recent performance, take a look at the next exercise.

In [ ]:
merged = merged.groupby('playerID').mean()

### Option 2 (Advanced): Analyzing Players' Most Recent Performance

In this approach, we aim to concentrate on the latest performance metrics of the players. By focusing on their most recent statistics, we can get insights into the current form and capabilities of each player. This method is particularly useful for analyses where recent performance is more relevant than historical data.

To implement this strategy, we will identify and extract the rows corresponding to each player's most recent year of play. This requires two steps in our code:

1. **Identifying the Most Recent Year for Each Player:** We'll group our data by each player and then find the index of the row with the most recent year for that player. This is done using the `.groupby()` method followed by `.idxmax()`, which gives us the index of the maximum value in a specified column within each group.

2. **Selecting Rows Based on Identified Indices:** With the indices of the most recent year for each player obtained, we then use `.loc[]` to select these specific rows from our DataFrame.

Here's how you can apply this method, with explanations on filling in the missing values:

```python
playersMostRecentYearIndices = merged.groupby('COLUMN_TO_GROUP')['COLUMN_OF_INTEREST'].idxmax()

# Now, use the indices to select the rows corresponding to each player's most recent performance.
merged = merged.loc[playersMostRecentYearIndices]
```

<!-- - **`'playerID'`** is used as the `COLUMN_TO_GROUP` because our goal is to analyze data on a per-player basis.
- **`'yearID'`** is chosen as the `COLUMN_OF_INTEREST` since we're looking for the most recent year, which is represented by the maximum value of `yearID` for each player. -->


This process filters our dataset to only include the latest available statistics for each player. It effectively simplifies the dataset, making it more manageable and focused on current performance levels.

Remember, this approach highlights the players' capabilities based on their last recorded performance, which may not always reflect their career-long trends or potential future performance.

In [ ]:
playersMostRecentYearIndices = merged.groupby('playerID')['yearID'].idxmax()
merged = merged.loc[playersMostRecentYearIndices]
merged.head()

,yearID,CompleteGames,Shutouts,Saves,OutsPitched,HitsAsPitcher,EarnedRuns,HomeRunsAsPitcher,WalksAsPitcher,StrikeoutsAsPitcher,...,SluggingPercentage,OpponentsBattingAverage,ZoneRating,Position_1B,Position_2B,Position_3B,Position_C,Position_OF,Position_P,Position_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aardsda01,2009.333333,0.000000,0.000000,0.122828,3.080773,1.032253,0.597564,0.225529,0.639950,0.993802,...,NaN,0.257444,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
aaronha01,1974.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.464469,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
aasedo01,1983.538462,0.063752,0.018489,0.161176,8.290755,2.679695,1.123944,0.219588,1.090490,1.554422,...,NaN,0.250846,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
abadan01,2002.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.5,0.0,0.0
abadfe01,2014.800000,0.000000,0.000000,0.003838,2.640671,0.883906,0.402578,0.109450,0.308937,0.691888,...,NaN,0.275400,0.0,0.0,0.0,0.0,0.0,0.0,1.1,0.0


## Getting rid of missing data

Now we have the dataset we can work with! Unfortunately, though, our nearest neighbors model will not be able to work with those `NaN`s, so we must "impute" some number to replace those. We can do so using code that looks like this:

```python
merged = merged.fillna(VALUE)
```

What value might make sense to fill in where we have missing statistics? Fill in the code cell below!

In [ ]:
merged = merged.fillna(0)

### (Optional) Focus on recent players

Now that we've merged our datasets, we may want to focus only on those players that have played recently, since they would be the best options to replace a player on our team.

Discuss with your teammates what you'd prefer your cutoff year to be for those you consider to be on your team, and alter the dataset to only include the recent players.

In [ ]:
merged = merged[merged['yearID'] >= 2019]
merged = merged.reset_index()
merged.head()

,playerID,yearID,CompleteGames,Shutouts,Saves,OutsPitched,HitsAsPitcher,EarnedRuns,HomeRunsAsPitcher,WalksAsPitcher,...,SluggingPercentage,OpponentsBattingAverage,ZoneRating,Position_1B,Position_2B,Position_3B,Position_C,Position_OF,Position_P,Position_SS
0,abbotco01,2021.5,0.0,0.0,0.000000,8.214286,2.803571,1.803571,0.875000,1.566964,...,0.000000,0.2625,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0
1,abramcj01,2022.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.327044,0.0000,0.0,0.0,1.0,0.0,0.0,1.0,0.000000,2.0
2,abreual01,2021.0,0.0,0.0,0.011905,3.147908,1.341631,0.891414,0.312410,0.781746,...,0.000000,0.4840,0.0,0.0,0.0,0.0,0.0,0.0,1.666667,0.0
3,abreubr01,2020.5,0.0,0.0,0.017155,3.247266,0.692161,0.342789,0.041349,0.807986,...,0.000000,0.1725,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0
4,acevedo01,2021.5,0.0,0.0,0.028571,3.100000,0.807143,0.378571,0.214286,0.321429,...,0.000000,0.2155,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0


## Scaling the data

Before moving on from data processing to cluster analysis, we need to make sure our data is scaled! Cluster analysis relies on calculating distances between different data points, so we want to make sure all of our features are on the same scale.

Below we use the describe method to take a look at the scale of our data. Take a look and discuss which features might have outsize influence on the clustering if we weren't to scale.

In [ ]:
merged.describe()

,yearID,CompleteGames,Shutouts,Saves,OutsPitched,HitsAsPitcher,EarnedRuns,HomeRunsAsPitcher,WalksAsPitcher,StrikeoutsAsPitcher,...,SluggingPercentage,OpponentsBattingAverage,ZoneRating,Position_1B,Position_2B,Position_3B,Position_C,Position_OF,Position_P,Position_SS
count,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,...,1449.000000,1449.000000,1449.0,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000,1449.000000
mean,2020.666839,0.000790,0.000383,0.009966,4.022803,1.431207,0.829686,0.236080,0.600972,1.158070,...,0.101754,0.181467,0.0,0.114223,0.126220,0.122917,0.080791,0.250930,0.636455,0.086150
std,1.009640,0.006199,0.003741,0.036770,4.617938,1.652143,1.041647,0.349385,0.699882,1.446616,...,0.176289,0.162671,0.0,0.309210,0.328254,0.316789,0.281101,0.438148,0.537681,0.269732
min,2019.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2020.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2020.500000,0.000000,0.000000,0.000000,3.000000,0.944030,0.444168,0.090909,0.400000,0.833333,...,0.000000,0.226600,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,2021.500000,0.000000,0.000000,0.000000,5.785714,2.333333,1.390806,0.382353,1.000000,1.645833,...,0.261146,0.288000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,1.000000,0.000000
max,2022.000000,0.142857,0.100000,0.474365,18.584987,9.000000,7.500000,3.500000,4.000000,7.689282,...,0.599481,0.934000,0.0,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000


To remedy the imbalance, we can use sklearn's `StandardScaler`! This will basically scale the data so that it all has the same average and spread. If you're wondering how it goes about doing this, take a look at the optional section of notebook 1 where we calculated $z$-scores!



In [ ]:
# Set the index of the dataset to be the playerID, so the data is only numerical
merged = merged.set_index("playerID")

# Scale the features
scaler = StandardScaler()
scaled_data = scaler.fit_transform(merged)

# Since StandardScaler gives us a numpy array, we make a dataframe on the fly for easier display
pd.DataFrame(scaled_data, columns=merged.columns).describe()

,yearID,CompleteGames,Shutouts,Saves,OutsPitched,HitsAsPitcher,EarnedRuns,HomeRunsAsPitcher,WalksAsPitcher,StrikeoutsAsPitcher,...,SluggingPercentage,OpponentsBattingAverage,ZoneRating,Position_1B,Position_2B,Position_3B,Position_C,Position_OF,Position_P,Position_SS
count,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03,...,1.449000e+03,1.449000e+03,1449.0,1449.000000,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03,1.449000e+03
mean,4.524132e-14,2.451838e-17,3.432574e-17,9.807353e-18,6.129596e-19,6.129596e-17,1.336252e-16,1.225919e-18,-1.287215e-17,1.164623e-16,...,-2.697022e-17,-6.497371e-17,0.0,0.000000,5.271452e-17,1.152364e-16,-7.355515e-18,-4.413309e-17,-7.110331e-17,-7.478107e-17
std,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00,...,1.000345e+00,1.000345e+00,0.0,1.000345,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00,1.000345e+00
min,-1.651494e+00,-1.275553e-01,-1.025045e-01,-2.711422e-01,-8.714261e-01,-8.665725e-01,-7.967884e-01,-6.759347e-01,-8.589726e-01,-8.008135e-01,...,-5.774007e-01,-1.115929e+00,0.0,-0.369531,-3.846523e-01,-3.881442e-01,-2.875091e-01,-5.729041e-01,-1.184114e+00,-3.195005e-01
25%,-6.607002e-01,-1.275553e-01,-1.025045e-01,-2.711422e-01,-8.714261e-01,-8.665725e-01,-7.967884e-01,-6.759347e-01,-8.589726e-01,-8.008135e-01,...,-5.774007e-01,-1.115929e+00,0.0,-0.369531,-3.846523e-01,-3.881442e-01,-2.875091e-01,-5.729041e-01,-1.184114e+00,-3.195005e-01
50%,-1.653033e-01,-1.275553e-01,-1.025045e-01,-2.711422e-01,-2.215612e-01,-2.949777e-01,-3.702313e-01,-4.156473e-01,-2.872503e-01,-2.245575e-01,...,-5.774007e-01,2.775487e-01,0.0,-0.369531,-3.846523e-01,-3.881442e-01,-2.875091e-01,-5.729041e-01,6.763682e-01,-3.195005e-01
75%,8.254906e-01,-1.275553e-01,-1.025045e-01,-2.711422e-01,3.818848e-01,5.462224e-01,5.388723e-01,4.188033e-01,5.703332e-01,3.372921e-01,...,9.044618e-01,6.551283e-01,0.0,-0.369531,-3.846523e-01,-3.881442e-01,-2.875091e-01,1.881369e-01,6.763682e-01,-3.195005e-01
max,1.320888e+00,2.292403e+01,2.663911e+01,1.263423e+01,3.154484e+00,4.582780e+00,6.405835e+00,9.345128e+00,4.858251e+00,4.516380e+00,...,2.824326e+00,4.627709e+00,0.0,6.100801,5.710295e+00,5.927390e+00,6.829815e+00,3.993342e+00,4.397332e+00,7.097831e+00


### 💡 Discussion Question

Discuss the results! What's changed about the data?

# Milestone 4: Cluster Analysis!

## Nearest Neighbors

Phew, that was a lot of data processing! Now that we've finished that up, we've provided the code below for you to explore the most similar players based on a K Nearest Neighbors model!

Feel free to adjust the number of neighbors produced by the model as well as the player in particular you are interested in. In order to better visualize the results, you can also choose two features to plot for the players. Take a look at how the nearest neighbors are influenced by the features you choose!

If you're confused by any particular aspect, let your instructor know!

In [ ]:
#@title Run this to explore the nearest neighbors model!

# Define ipywidgets for user input
chosen_player_id_widget = widgets.Dropdown(options=merged.index, value='abbotco01', description='Player ID:')
number_of_neighbors_widget = widgets.IntSlider(value=5, min=1, max=20, step=1, description='Number of Neighbors:')

features = merged.columns
first_feature_widget = widgets.Dropdown(options=features, value='OnBasePercentage', description='First Feature:')
second_feature_widget = widgets.Dropdown(options=features, value='BattingAverage', description='Second Feature:')

out = Output()

# Define a function to execute when the user clicks the button
def on_button_clicked(b):
    global merged
    out.clear_output()
    # Check if playerID column exists and set it as index for easy searching
    if "playerID" in merged.columns:
        merged = merged.set_index("playerID")

    chosen_player_id = chosen_player_id_widget.value
    number_of_neighbors = number_of_neighbors_widget.value
    first_feature_to_compare = first_feature_widget.value
    second_feature_to_compare = second_feature_widget.value

    chosen_player_stats = merged.loc[chosen_player_id]
    target_player_features = chosen_player_stats.to_numpy().flatten()

    # Fit the KNN model
    nbrs = NearestNeighbors(n_neighbors=number_of_neighbors)
    nbrs.fit(scaled_data)

    # Find the nearest neighbors of the target player
    neighbors_indices = nbrs.kneighbors([target_player_features], return_distance=False)
    nearest_neighbors = merged.iloc[neighbors_indices[0]]

    with out:
      display(nearest_neighbors)

      # Visualization
      plt.figure(figsize=(10, 6))

      sns.scatterplot(data=merged.reset_index(), x=first_feature_to_compare, y=second_feature_to_compare, s=100, color='gray', alpha=0.5, label='Other Players')

      sns.scatterplot(data=nearest_neighbors.reset_index(), x=first_feature_to_compare, y=second_feature_to_compare, s=150, color='blue', label='Nearest Neighbors')

      plt.scatter(chosen_player_stats[first_feature_to_compare], chosen_player_stats[second_feature_to_compare], color='red', s=200, label='Chosen Player')

      # Label the points with player IDs
      for idx, row in nearest_neighbors.iterrows():
          plt.text(row[first_feature_to_compare], row[second_feature_to_compare], idx, horizontalalignment='right')
      plt.text(chosen_player_stats[first_feature_to_compare], chosen_player_stats[second_feature_to_compare], chosen_player_id, horizontalalignment='right', color='black')

      plt.title('Nearest Neighbors in Feature Space')
      plt.xlabel(first_feature_to_compare)
      plt.ylabel(second_feature_to_compare)
      plt.legend()
      plt.show()

# Create a button widget
button = widgets.Button(description='Find Neighbors')

# Attach the function to the button's click event
button.on_click(on_button_clicked)

# Display the widgets
display(chosen_player_id_widget, number_of_neighbors_widget, first_feature_widget, second_feature_widget, button)

# Display separate output for plots so we can clear on button press
display(out)


Dropdown(description='Player ID:', options=('abbotco01', 'abramcj01', 'abreual01', 'abreubr01', 'acevedo01', '…

IntSlider(value=5, description='Number of Neighbors:', max=20, min=1)

Dropdown(description='First Feature:', index=42, options=('yearID', 'CompleteGames', 'Shutouts', 'Saves', 'Out…

Dropdown(description='Second Feature:', index=41, options=('yearID', 'CompleteGames', 'Shutouts', 'Saves', 'Ou…

Button(description='Find Neighbors', style=ButtonStyle())

Output()

### 💡 Discussion Question

- How might this cluster analysis be helpful for us in choosing players for our team? How can we combine this with the work we did in the previous notebook?
- *(Optional)* You may notice a lot of players' data clustered at a singular point, depending on your choice of features. Which of our previous steps in data processing might be leading to this?



# Congratulations on completing NB3!

That should conclude our baseball analysis! We hope you've had a blast and have learned plenty about the ins and outs of baseball and data analytics!